In [1]:
# !pip install -Uqq fastbook nbdev ipywidgets

In [12]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
import fastai
print(f'fastai version: {fastai.__version__}')

import warnings
warnings.filterwarnings("ignore", message="The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.")
warnings.filterwarnings("ignore", message="Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.")

# BASE_DIR='/content/gdrive/MyDrive/Colab Notebooks'
BASE_DIR='/home/tabrez/MEGA/MEGA/code/fastai-book'
#!ln -sf '/content/gdrive/MyDrive/Colab Notebooks' ~/.fastai

fastai version: 2.7.12


In [2]:
path = untar_data(URLs.PETS)
# !ls -l ~/.fastai/data/oxf*/images | head
path , len((path/'images').ls()), len(get_image_files(path/'images'))

(Path('/home/tabrez/.fastai/data/oxford-iiit-pet'), 7393, 7390)

In [3]:
def label_func(fname):
  return'_'.join(fname.name.split('_')[:-1])

db = DataBlock(blocks=(ImageBlock, CategoryBlock),
               get_items = get_image_files,
               splitter = RandomSplitter(valid_pct=0.2, seed=42),
               get_y = label_func,
               item_tfms=Resize(446),
               batch_tfms=aug_transforms(size=244, min_scale=0.75))
# db.summary(path/'images')
dls = db.dataloaders(path/'images')

In [4]:
len(dls.train), len(dls.train_ds), len(dls.valid), len(dls.valid_ds)
x, y = dls.one_batch()
len(x), len(y), x.shape, y.shape

# Learner.save and Learner.load use the default path based on following fields
# learn.path, learn.model_dir

(64, 64, torch.Size([64, 3, 244, 244]), torch.Size([64]))

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)

import time
start_time = time.time()
## Fine tune the model if no saves are available to load
# learn.fine_tune(2)## Save the model & weights so we can load it later without incurring processing costs again and again
learn.save(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet'))
print("--- %s seconds ---" % (time.time() - start_time))

## Save the model & weights so we can load it later without incurring processing costs again and again
#learn.save(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet'))

## Load the model & weights from the file if they've been saved before
learn.load(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet'))

In [17]:
## The Learning Rate Finder
learn2 = vision_learner(dls, resnet34, metrics=error_rate)
#learn2.fine_tune(2, base_lr=0.1)

epoch,train_loss,valid_loss,error_rate,time
0,2.681761,6.949239,0.598782,00:17


epoch,train_loss,valid_loss,error_rate,time
0,4.155255,3.893214,0.880244,00:20
1,2.666315,1.881500,0.566982,00:20


In [18]:
#learn2.save(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet-lr-find'))
learn2.load(file=Path(f'{BASE_DIR}/saves/oxford-iiit-pet-lr-find'))

Path('/home/tabrez/MEGA/MEGA/code/fastai-book/saves/oxford-iiit-pet-lr-find.pth')

In [ ]:
lr_finder = learn2.lr_find()
lr_finder

In [19]:
lr_min = lr_finder.lr_min
lr_steep = lr_finder.lr_steep
print(f'min: {lr_min}, steep: {lr_steep}')

NameError: name 'lr_finder' is not defined